In [8]:
# Importar Librerías.
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import webbrowser
import datetime
from datetime import datetime, timedelta
import lxml
import requests
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from unidecode import unidecode

In [9]:
# Importar DataFrame 'DTH'.
df_hogar_dth = pd.read_excel("data_hogar.xlsx")

#### Ingresar a CE

In [ ]:
# Conectar con el navegador.
driver = webdriver.Chrome()
# Maximizar el navegador.
driver.maximize_window()
# Ingresar a la web.
driver.get("Ingresa tu URL")
# Dar clic en "Televenta".
enlace_televenta = driver.find_element("xpath", "//a[@href='indexTLV.html']")
enlace_televenta.click()
# Ingresar usuario y contraseña.
ingreso_usuario = driver.find_element("id", "username")
ingreso_contrasena = driver.find_element("id", "password")
ingreso_usuario.send_keys("Ingresa tu usuario")
ingreso_contrasena.send_keys("Ingresa tu contraseña")
formulario = driver.find_element("id", "conectar")  
formulario.click()

#### DTH

In [12]:
# Esperar 3 segundos.
driver.implicitly_wait(3)

In [ ]:
# Esperar 10 segundos.
wait = WebDriverWait(driver, 10)  
# Iterar sobre el DataFrame.
for index, row in df_hogar_dth.iterrows():
    # Seleccionar 'DHT'.
    dth = driver.find_element("xpath", "//a[@href='#dth']")
    dth.click()
    # Seleccionar "Nuevo Contrato".
    boton = driver.find_element("xpath", "//a[@onclick='formularioProspeccion(\"DTH\")']")
    boton.click()

    # Seleccionar 'DTH'.
    time.sleep(2)
    producto_preferencia = driver.find_element("id", "checkdth")  
    producto_preferencia.click()
    
    # Ingresar 'Comuna'.
    # Normalizar la comuna antes de ingresarla
    comuna_normalizada = unidecode(row['COMUNA']).replace('ñ', 'n')

    # Ingresar 'Comuna'.
    comuna_input = wait.until(EC.element_to_be_clickable((By.ID, 'prosp_comuna')))
    comuna_input.clear()
    comuna_input.send_keys(comuna_normalizada)
    time.sleep(2)
    comuna_input.send_keys(Keys.TAB) 
    comuna_input.send_keys(Keys.ENTER) 

    # Ingresar 'Calle'.
    calle_input = wait.until(EC.element_to_be_clickable((By.NAME, 'prosp_calle')))
    calle_input.clear()
    calle_input.send_keys(row['DIRECCIÓN']) 
    time.sleep(2)
    calle_input.send_keys(Keys.TAB) 
    calle_input.send_keys(Keys.ENTER)      
    
    # Ingresar 'Número'.
    numero_input = wait.until(EC.element_to_be_clickable((By.NAME, 'prosp_numero')))
    numero_input.clear()
    numero_input.send_keys(row['NÚMERO EXTERIOR']) 
    time.sleep(2) 
    numero_input.send_keys(Keys.TAB) 
    numero_input.send_keys(Keys.ENTER) 

    # Ingresar 'Depto.'.
    try:
        if pd.notna(row['DEPTO']) and row['DEPTO'] != '':  
            depto_input = wait.until(EC.element_to_be_clickable((By.NAME, 'prosp_depto')))
            depto_input.clear()
            depto_input.send_keys(row['DEPTO'])  
            
            depto_input.send_keys(Keys.TAB) 
            depto_input.send_keys(Keys.ENTER)
            time.sleep(2)
        else: next
    except:
        next    


    # Ingresar 'Interior'.
    if pd.notna(row['NÚMERO INTERIOR']) and row['NÚMERO INTERIOR'] != '':
        try:
            # Convertir el valor a entero
            numero_interior = int(row['NÚMERO INTERIOR'])
            
            # Localizar el campo de entrada y enviar el valor
            interior_input = wait.until(EC.element_to_be_clickable((By.NAME, 'prosp_interior')))
            interior_input.clear()
            interior_input.send_keys(str(numero_interior))  # Enviar el valor como cadena
            
            interior_input.send_keys(Keys.TAB)
            interior_input.send_keys(Keys.ENTER)
            time.sleep(2)
        except ValueError:
            print(f"El valor '{row['NÚMERO INTERIOR']}' no es un número entero válido.")
    else:
        next


    # Ingresar 'Rut'.
    rut_input = wait.until(EC.element_to_be_clickable((By.NAME, 'prosp_rut')))
    rut_input.clear()
    rut_input.send_keys(row['NRO DOCUMENTO'])  
    rut_input.send_keys(Keys.TAB) 
    rut_input.send_keys(Keys.ENTER) 
    time.sleep(2)

    try:
        # Seleccionar 'Consultar'.
        consultar = driver.find_element(By.ID, "consultar")  
        consultar.click()
        time.sleep(2)
        # Buscar el cuadro emergente y obtener su texto
        alert_text_element = driver.find_element(By.XPATH, "//b[contains(text(), 'Cliente tiene una venta en proceso en CE')]")
        
        if alert_text_element:
            #print("Se detectó la frase en el cuadro emergente. Regresando a la página anterior...")
            id_cliente_1 = row["ID DE LA ORDEN"] 
            print(f"El ID {id_cliente_1} tiene una venta en proceso en CE.")
            boton_aceptar = driver.find_element(By.XPATH, "//button[span[text()='Aceptar']]")
            boton_aceptar.click()
            boton_volver = driver.find_element(By.XPATH, "//button[span[text()='Volver']]")
            boton_volver.click()
            
    except:
        next
        # Esperar 20 segundos.
        wait2 = WebDriverWait(driver,20)  
        # Esperar hasta que la ventana emergente cargue la información
        wait2.until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-dialog-content')))


        # Seleccionar 'Aceptar'.
        button_container = wait2.until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-dialog-buttonset')))
        button = button_container.find_element(By.CLASS_NAME, 'ui-button-text')
        time.sleep(2)
        button.click()

        # Esperar 12 segundos.
        driver.implicitly_wait(12)

        #############################
        #           CLIENTE         #
        #############################
        

        # Ingresar 'Nombre'.
        nombre_input = wait.until(EC.element_to_be_clickable((By.NAME, 'CL_NOMBRE')))
        nombre_input.clear()
        nombre_input.send_keys(row['NOMBRE DEL CLIENTE CE']) 

        # Ingresar 'Teléfono Particular'.
        fono_particular_input = wait.until(EC.element_to_be_clickable((By.NAME, 'CL_FONO_PARTICULAR_NUMERO')))
        fono_particular_input.clear()
        fono_particular_input.send_keys(row['NÚMERO DE TELÉFONO']) 

        # Ingresar 'Teléfono Oficina'.
        if pd.notna(row['TELÉFONO DE CONTACTO']) and row['TELÉFONO DE CONTACTO'] != '': 
            fono_oficina_input = wait.until(EC.element_to_be_clickable((By.NAME, 'CL_FONO_OFICINA_NUMERO')))
            fono_oficina_input.clear()
            fono_oficina_input.send_keys(row['TELÉFONO DE CONTACTO'])
        else: 
            fono_oficina_input = wait.until(EC.element_to_be_clickable((By.NAME, 'CL_FONO_OFICINA_NUMERO')))
            fono_oficina_input.clear()
            fono_oficina_input.send_keys(row['NÚMERO DE TELÉFONO']) 

        # Ingresar 'Teléfono Celular'.
        if pd.notna(row['TELEFONO RECIBE']) and row['TELEFONO RECIBE'] != '': 
            fono_celular_input = wait.until(EC.element_to_be_clickable((By.NAME, 'CL_FONO_CELULAR_NUMERO')))
            fono_celular_input.clear()
            fono_celular_input.send_keys(row['TELEFONO RECIBE'])
        else: 
            fono_oficina_input = wait.until(EC.element_to_be_clickable((By.NAME, 'CL_FONO_CELULAR_NUMERO')))
            fono_oficina_input.clear()
            fono_oficina_input.send_keys(row['NÚMERO DE TELÉFONO']) 

        #-------------------------------------------------------------------------------------------------------------------------------
        # Ingresar 'Email'.
        email1_input = wait.until(EC.element_to_be_clickable((By.NAME, 'CL_EMAIL')))
        email1_input.clear()
        email_modificado = "." + row['CORREO ELECTRÓNICO']
        email1_input.send_keys(email_modificado)    

        #############################
        #           CUENTA         #
        #############################

        # Seleccionar 'Boleta'.
        consultar = driver.find_element("id", "CU_DOCUMENTO_BOLETA")  
        consultar.click()

        # Ingresar 'Email'.
        email2_input = wait.until(EC.element_to_be_clickable((By.NAME, 'CU_EMAIL')))
        email2_input.clear()
        email2_input.send_keys(row['CORREO ELECTRÓNICO']) 

        ##############################
        #           INTERNET         #
        ##############################

        # Seleccionar 'Plan BAFI'.
        select_element = Select(driver.find_element(By.ID, "SE_IBA_PLAN"))
        select_element.select_by_index(1)
        select_element.select_by_value("PLAN BAFI 100GB")

        ######################################
        #           VALORES SERVICIO         #
        ######################################

        # Seleccionar 'Efectivo'.
        efectivo = driver.find_element("id", "VA_EFECTIVO_1")  
        efectivo.click()

        # Ingresar 'Observaciones'.
        obs_input = wait.until(EC.element_to_be_clickable((By.NAME, 'VA_OBSERVACIONES')))
        obs_input.clear()
        obs_input.send_keys(row['OBSERVACIONES']) 

        ##################################################
        #           OBSERVACIONES DE INSTALACIÓN         #
        ##################################################

        # Ingresar 'Observaciones'.
        obs2_input = wait.until(EC.element_to_be_clickable((By.NAME, 'OBS_INSTALACION')))
        obs2_input.clear()
        obs2_input.send_keys(row['OBSERVACIONES']) 

        ########################################
        #           FIRMAS AUTORIZADAS         #
        ########################################

        # Seleccionar 'Nombre2'.
        elemento_nombre2 = driver.find_element(By.XPATH, "//*[@onclick='AutoNombreadicionaldos();']")
        elemento_nombre2.click()
        time.sleep(1)

        # Seleccionar 'Rut2'.
        elemento_rut2 = driver.find_element(By.XPATH, "//*[@onclick='AutoRutadicionaldos();']")
        elemento_rut2.click()
        time.sleep(1)

        # Seleccionar 'Nombre1'.
        elemento_nombre1 = wait.until(EC.element_to_be_clickable((By.NAME, 'firmaNombreUno')))
        elemento_nombre1.clear()
        elemento_nombre1.send_keys(row['NOMBRE RECIBE CE']) 
        time.sleep(1)
        # Seleccionar 'Rut1'.
        elemento_rut1 = wait.until(EC.element_to_be_clickable((By.NAME, 'firmaRutUno')))
        elemento_rut1.clear()
        elemento_rut1.send_keys(row['RUT RECIBE CE']) 
        time.sleep(1)

        # Seleccionar 'Digito Rut1'.
        try:
            elemento_digito_rut1 = wait.until(EC.element_to_be_clickable((By.NAME, 'firmaDVUno')))
            elemento_digito_rut1.clear()
            elemento_digito_rut1.send_keys(row['DIGITO RECIBE CE']) 
        except:
            next    

        # Seleccionar Grabar.
        boton = driver.find_element(By.ID, "grabar")  
        print("Onclick:", boton.get_attribute("onclick"))
        boton.click()

        # Seleccionar Aceptar
        time.sleep(2)  
        alerta = driver.switch_to.alert  
        print("Texto de la alerta:", alerta.text)  

        # Acepta la alerta
        alerta.accept()

        # Chequear si se ingresó correctamente.
        id_cliente = row["ID DE LA ORDEN"] 
        print(f"El ID {id_cliente} se ingresó correctamente.")
    time.sleep(4)  


El ID 11655402910225 tiene una venta en proceso en CE.
El ID 11643004058475 tiene una venta en proceso en CE.
El ID 11643952371074 tiene una venta en proceso en CE.


#### Firmar Contratos

In [ ]:
try:
    # Espera a que se carguen todos los botones "Firmar" visibles en la página
    botones_firmar = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, "//input[@value='Firmar']"))
    )

    print(f"Se encontraron {len(botones_firmar)} contratos con botón 'Firmar'.")

    # Itera por cada botón y realiza clic
    for i, boton in enumerate(botones_firmar):
        try:
            # Desplázate hasta el botón (por si no está visible en la ventana)
            driver.execute_script("arguments[0].scrollIntoView();", boton)

            # Haz clic en el botón "Firmar"
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable(boton))
            boton.click()

            #print(f"Contrato {i + 1}: Se hizo clic en 'Firmar'.")

            # Espera a que aparezca el cuadro emergente con el botón "Continuar"
            boton_continuar = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[span[text()='Continuar']]"))
            )

            # Haz clic en el botón "Continuar"
            boton_continuar.click()
            #print(f"Contrato {i + 1}: Se hizo clic en 'Continuar'.")

            # Espera un momento para que el sistema procese antes de continuar con el siguiente contrato
            time.sleep(2)

                # Seleccionar Aceptar
            time.sleep(2)  
            alerta = driver.switch_to.alert  
            #print("Texto de la alerta:", alerta.text)  

            # Acepta la alerta
            alerta.accept()

        except Exception as e:
            print(f"Error al intentar firmar el contrato {i + 1}: {e}")

except Exception as e:
    print(f"Error general al procesar los contratos: {e}")

# Cierra el navegador
driver.quit()

Se encontraron 2 contratos con botón 'Firmar'.
Contrato 1: Se hizo clic en 'Firmar'.
Contrato 1: Se hizo clic en 'Continuar'.
Texto de la alerta: Desea continuar la firma de contrato por Autenticación vía: Preguntas?
Contrato 2: Se hizo clic en 'Firmar'.
Contrato 2: Se hizo clic en 'Continuar'.
Texto de la alerta: Desea continuar la firma de contrato por Autenticación vía: Preguntas?
